In [1]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from superres import superres_metrics, ssim, psnr
from superres import RRDB_Net, MultiImageToMultiChannel, TwoXModel, TwoYLoss
import czifile
from superres.helpers import get_czi_shape_info, build_index
import imageio

datasetname = 'multiframe_001'
data_path = Path('/scratch/bpho')
datasets = data_path/'datasets'
datasources = data_path/'datasources'
dataset = datasets/datasetname

hr_path = dataset/'hr'
lr_path = dataset/'lr'
lr_up_path = dataset/'lr_up'

In [2]:
class NpyRawImageList(ImageList):
    def open(self, fn):
        img_data = np.load(fn)
        return Image(tensor(img_data[None]))
    
    def analyze_pred(self, pred:Tensor):
        if len(pred.shape) > 3: pred = pred[0]
        return pred
    
    def reconstruct(self, t: torch.Tensor):
        return Image(t.float().clamp(min=0,max=1))


class TransformableLists(ItemBase):
    def __init__(self, img_lists):
        self.img_lists = img_lists
       
    def __repr__(self):
        return f'MultiImg: {len(self.img_lists)}'
    
    @property
    def data(self):
        img_data = torch.stack([torch.stack([img.data for img in img_list]) 
                                            for img_list in self.img_lists])
        data = tensor(img_data)
        return data
    
    def apply_tfms(self, tfms, **kwargs):
        first_time = True
        save_img_lists = []
        for img_list in self.img_lists:
            save_img_list = []
            for img in img_list:
                new_img = img.apply_tfms(tfms, do_resolve=first_time, **kwargs)
                first_time = False
                save_img_list.append(new_img)
            save_img_lists.append(save_img_list)
        self.img_lists = save_img_lists
        return self
    
class MultiImageDataBunch(ImageDataBunch):
    def batch_stats(self, funcs:Collection[Callable]=None)->Tensor:
        "Grab a batch of data and call reduction function `func` per channel"
        funcs = ifnone(funcs, [torch.mean,torch.std])
        ds_type = DatasetType.Valid if self.valid_dl else DatasetType.Train
        x = self.one_batch(ds_type=ds_type, denorm=False)[0].cpu()
        def multi_channel_view(x):
            return x.transpose(3,0).contiguous().view(x.shape[3],-1)
        return [func(multi_channel_view(x), 1) for func in funcs]

    def normalize(self, stats:Collection[Tensor]=None, do_x:bool=True, do_y:bool=False)->None:
        "Add normalize transform using `stats` (defaults to `DataBunch.batch_stats`)"
        if getattr(self,'norm',False): raise Exception('Can not call normalize twice')
        if stats is None: self.stats = self.batch_stats()
        else:             self.stats = stats
        self.norm,self.denorm = normalize_funcs(*self.stats, do_x=do_x, do_y=do_y)
        self.add_tfm(self.norm)
        return self
    
class MultiImageList(ItemList):
    _bunch = MultiImageDataBunch
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def open(self, fn):
        img_data = np.load(fn)
        img_lists = []
        for j in range(img_data.shape[0]):
            imgs = []
            for i in range(img_data.shape[1]):
                imgs.append(Image(tensor(img_data[j,i,:,:][None])))
            img_lists.append(imgs)
        return TransformableLists(img_lists)
    
    def get(self, i):
        fn = super().get(i)
        img_lists = self.open(fn)
        return img_lists    
    

class MultiToMultiImageList(MultiImageList):
    _label_cls = NpyRawImageList

In [3]:
def map_to_hr(x):
    hr_name = x.relative_to(lr_path)
    return hr_path/hr_name

def get_src(size=128, scale=1):
    if scale != 1: use_lr_path = lr_path
    else: use_lr_path = lr_up_path
        
    def map_to_hr(x):
        hr_name = x.relative_to(use_lr_path)
        return hr_path/hr_name
    
    src = (MultiToMultiImageList.from_folder(use_lr_path, extensions=['.npy'])
           .split_by_folder()
           .label_from_func(map_to_hr))
    return src

def get_data(bs, size, scale=1, max_zoom=1.):
    src = get_src(size, scale=scale)    
    tfms = get_transforms(flip_vert=True, max_zoom=max_zoom)
    data = (src
            .transform(tfms, size=size)
            .transform_y(tfms, size=size*scale)
            .databunch(bs=bs).normalize(do_y=True))
    return data

In [ ]:
print('cool')

In [ ]:
bs = 8
size = 128
data = get_data(bs, size, scale=4)

In [4]:
bs = 4
size = 256
data = get_data(bs, size, scale=4)

in_c = 3
out_c = 1
nf = gcval = 16
nb = 5
loss = TwoYLoss(stable_wt=0.0)
model = TwoXModel(MultiImageToMultiChannel(RRDB_Net(3, 1, nf, nb, gc=gcval )))
learn = Learner(data, model, callback_fns=LossMetrics, loss_func=loss)
#learn = learn.load('rddb.1')
gc.collect()

7

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, max_lr=1e-3)

In [ ]:
learn.save('rddb.1')


In [5]:
movie_files = list(Path('/scratch/bpho/datasets/movies_001/test').glob('*.czi'))
#movie_files += list(Path('/scratch/bpho/datasources/low_res_test/').glob('low res confocal*.czi'))

In [6]:
def image_from_tiles(learn, in_img, tile_sz=128, scale=4):
    cur_size = in_img.shape[1:3]
    c = in_img.shape[0]
    new_size = (cur_size[0]*scale, cur_size[1]*scale)
    w, h = cur_size
    
    in_tile = torch.zeros((c,tile_sz,tile_sz))
    out_img = torch.zeros((1,w*scale,h*scale))
    
    for x_tile in range(math.ceil(w/tile_sz)):
        for y_tile in range(math.ceil(h/tile_sz)):
            x_start = x_tile

            x_start = x_tile*tile_sz
            x_end = min(x_start+tile_sz, w)
            y_start = y_tile*tile_sz
            y_end = min(y_start+tile_sz, h)
            
            
            in_tile[:,0:(x_end-x_start), 0:(y_end-y_start)] = tensor(in_img[:,x_start:x_end, y_start:y_end])
            
            img_list = [Image(in_tile[i][None]) for i in range(3)]
            tlist = TransformableLists([img_list, img_list])
            out_tile,_,_ = learn.predict(tlist)
            
            out_x_start = x_start * scale
            out_x_end = x_end * scale
            out_y_start = y_start * scale
            out_y_end = y_end * scale

            #print("out: ", out_x_start, out_y_start, ",", out_x_end, out_y_end)
            in_x_start = 0
            in_y_start = 0
            in_x_end = (x_end-x_start) * scale
            in_y_end = (y_end-y_start) * scale
            #print("tile: ",in_x_start, in_y_start, ",", in_x_end, in_y_end)
           
            out_img[:,out_x_start:out_x_end, out_y_start:out_y_end] = out_tile.data[:,
                                                                                  in_x_start:in_x_end, 
                                                                                  in_y_start:in_y_end]
    return out_img

In [20]:
def czi_predict_movie(learn, czi_in, orig_out='orig.tif', pred_out='pred.tif', size=128, wsize=3):
    with czifile.CziFile(czi_in) as czi_f:
        proc_axes, proc_shape = get_czi_shape_info(czi_f)
        channels = proc_shape['C']
        depths = proc_shape['Z']
        times = proc_shape['T']
        x,y = proc_shape['X'], proc_shape['Y']
        
        
        data = czi_f.asarray()
        preds = []
        origs = []
        img_max = None
        for t in progress_bar(list(range(0,times-wsize+1))):
            idx = build_index(proc_axes, {'T': slice(t,t+wsize), 'C': 0, 'Z':0, 'X':slice(0,x),'Y':slice(0,y)})
            img = data[idx].astype(np.float32)
            img /= img.max()
            
            out_img = image_from_tiles(learn, img, tile_sz=size)
            pred = (out_img*255).cpu().numpy().astype(np.uint8)
            preds.append(pred)
            orig = (img[1][None]*255).astype(np.uint8)
            origs.append(orig)
            
        all_y = np.concatenate(preds)
        #print(all_y.shape)
        imageio.mimwrite(pred_out, all_y) #, fps=30, macro_block_size=None) # for mp4
        all_y = np.concatenate(origs)
        #print(all_y.shape)
        imageio.mimwrite(orig_out, all_y) #, fps=30, macro_block_size=None)

In [15]:
bs = 1
size = 512
data = get_data(bs, size, scale=4)

loss = TwoYLoss()
model = TwoXModel(MultiImageToMultiChannel(RRDB_Net(3, 1, nf, nb, gc=gcval )))
learn = Learner(data, model, callback_fns=LossMetrics, loss_func=loss)
gc.collect()
learn = learn.load('rddb.1')

In [21]:
for fn in movie_files:
    pred_name = f'{fn.stem}_pred.tif'
    orig_name = f'{fn.stem}_orig.tif'
    czi_predict_movie(learn, fn, size=size, orig_out=orig_name, pred_out=pred_name )
    break